# GGHM Traffic Re-assignment
### Taken from an older GGHM toolbox

This script looks to re-assign traffic onto a GGHM network and references the parameters used in the assignmnet spec from an older GGHM toolbox. Primarily uses a SOLA assignment spec for the 10 auto classes.

Developed for: QSBRT <br>
Developed by: Peter

In [2]:
# Dependencies
from __future__ import division
from os import path
import multiprocessing as _mpl
import contextlib as _ctx
from collections import defaultdict
# from inro.emme.matrix import MatrixData
# import inro.modeller as _m
# _modeller = _m.Modeller()

# # Modeller Tools
# _run_sola_assignment = _modeller.tool('inro.emme.traffic_assignment.sola_traffic_assignment')
# _do_network_calculation = _modeller.tool('inro.emme.network_calculation.network_calculator')

### Define methods

In [3]:
def _get_base_assignment_spec(iterations, normalized_gap, best_relative_gap, relative_gap,n_threads):
# Constructs a SOLA assignment specification skeleton
    spec = {
        "type": "SOLA_TRAFFIC_ASSIGNMENT",
        "classes": [],
        "path_analysis": {
            "link_component": "length",
            "turn_component": None,
            "operator": "+",
            "selection_threshold": {
                "lower": None,
                "upper": None
            },
            "path_to_od_composition": {
                "considered_paths": "ALL",
                "multiply_path_proportions_by": {
                    "analyzed_demand": False,
                    "path_value": True
                }
            }
        },
        "cutoff_analysis": None,
        "traversal_analysis": None,
        "performance_settings": {
            "number_of_processors": n_threads
        },
        "background_traffic": None,
        "stopping_criteria": {
            "max_iterations": iterations,
            "relative_gap": relative_gap,
            "best_relative_gap": best_relative_gap,
            "normalized_gap": normalized_gap
        }
    }

    return spec


def _setup_assignment_class(mode_code, demand_mid, time_mid, link_exatt=None):
# Constructs a single class's component of the SOLA specification
    spec = {
        "mode": mode_code,
        "demand": demand_mid,
        "generalized_cost": None,
        "results": {
            "link_volumes": link_exatt,
            "turn_volumes": None,
            "od_travel_times": {
                "shortest_paths": time_mid
            }
        },
        "path_analysis": None,
        "cutoff_analysis": None,
        "traversal_analysis": None,
        "analysis": None
    }

    return spec

def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"

In [4]:
car_list ={
    "trips_am_peak_auto1_nt" :"mf121",
    "trips_am_peak_auto1_toll" :"mf122",
    "trips_am_peak_auto2_nt_hov" :"mf124",
    "trips_am_peak_auto2_nt_nh" :"mf123",
    "trips_am_peak_auto2_toll_hov" :"mf126",
    "trips_am_peak_auto2_toll_nh" :"mf125",
    "trips_am_peak_auto3_nt_hov" :"mf128",
    "trips_am_peak_auto3_nt_nh" :"mf127",
    "trips_am_peak_auto3_toll_hov" :"mf130",
    "trips_am_peak_auto3_toll_nh" :"mf129",}

modes = {
    'sov_mode':'j',
    'sovt_mode':'J',
    'hov2_mode':'h',
    'hov2t_mode':'H',
    'hov3_mode':'i',
    'hov3t_mode':'H',
}

In [5]:
iterations = 100
normalized_gap = 0.05
best_relative_gap = 0
relative_gap = 0
n_threads = _mpl.cpu_count()

spec = _get_base_assignment_spec(iterations, normalized_gap, best_relative_gap, relative_gap,n_threads)
spec['classes'] = [
    # SOV
    _setup_assignment_class(modes['sov_mode'],car_list['trips_am_peak_auto1_nt'],None,link_exatt="@volau1_nt"),
    _setup_assignment_class(modes['sovt_mode'],car_list['trips_am_peak_auto1_toll'],None,link_exatt="@volau1_toll"),
    # HOV
    _setup_assignment_class(modes['sov_mode'],car_list['trips_am_peak_auto2_nt_nh'],None,link_exatt="@volau2_nt_nh"),
    _setup_assignment_class(modes['hov2_mode'],car_list['trips_am_peak_auto2_nt_hov'],None,link_exatt="@volau2_nt_hov"),
    _setup_assignment_class(modes['sovt_mode'],car_list['trips_am_peak_auto2_toll_nh'],None,link_exatt="@volau2_toll_nh"),
    _setup_assignment_class(modes['hov2t_mode'],car_list['trips_am_peak_auto2_toll_hov'],None,link_exatt="@volau2_toll_hov"),
    # HOV3+
    _setup_assignment_class(modes['sov_mode'],car_list['trips_am_peak_auto3_nt_nh'],None,link_exatt="@volau3_nt_nh"),
    _setup_assignment_class(modes['hov3_mode'],car_list['trips_am_peak_auto3_nt_hov'],None,link_exatt="@volau3_nt_hov"),
    _setup_assignment_class(modes['sovt_mode'],car_list['trips_am_peak_auto3_toll_nh'],None,link_exatt="@volau3_toll_nh"),
    _setup_assignment_class(modes['hov3t_mode'],car_list['trips_am_peak_auto3_toll_hov'],None,link_exatt="@volau3_toll_hov")
]

In [8]:
dir = 
for matrix_name in matrix_list.keys():
        matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
        create_matrix(matrix_id = matrix_list[matrix_name],
                      matrix_name= matrix_name,
                      matrix_description = matrix_name)

    import_matrices_from_directory(dir, car_list, ".mdf", scenario)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)